In [22]:
import pickle
import numpy as np
from classes_colors import *
import plotly.graph_objects as go

In [4]:
with open("sensor_data_filtered.pkl", "rb") as f:
    sensor_data = pickle.load(f)

with open("interpolation_functions.pkl", "rb") as f:
    interp_funcs = pickle.load(f)

with open("sensor_labels.pkl", "rb") as f:
    labels = pickle.load(f)

In [15]:
class SensorCls:
    def __init__(self, day, mat, sensor, data, labels, interp_funcs):
        self._data = data[f"day{day}"][f"mat{mat}"][sensor]
        self._labels = labels[f"day{day}"][f"mat{mat}"]
        self._interp_funcs = interp_funcs[f"day{day}"][f"mat{mat}"][sensor]

        self._raw_cls_data_list = self._build_raw_cls_data_list()

    def _build_raw_cls_data_list(self):
        cls_data_list = []
        for label in self._labels:
            start = label["start"]
            end = label["end"]
            cls = label["label"]
            heater_data_list = []
            for i in range(10):
                heater_step = self._data[i]
                time_data = heater_step["Time Since PowerOn"].values
                filt_data = heater_step["Filtered"].values

                mask = (time_data >= start) & (time_data <= end)
                time_cls = time_data[mask]
                filt_cls = filt_data[mask]
                heater_data_list.append({
                    "start": time_cls[0],
                    "end": time_cls[-1],
                    "num_samples": len(time_cls),
                    "sample_times": time_cls,
                    "sample_vals": filt_cls
                })

            cls_data_list.append({
                "class": cls,
                "heater_data_list": heater_data_list
            })

        return cls_data_list

    def get_interpolated_data(self):
        cls_data_list = []
        for raw_cls_data in self._raw_cls_data_list:
            cls = raw_cls_data["class"]
            heater_data_list = raw_cls_data["heater_data_list"]
            
            start = max([el["start"] for el in heater_data_list])
            end = min([el["end"] for el in heater_data_list])
            num_samples = max([el["num_samples"] for el in heater_data_list])
            
            sample_times = np.linspace(start, end, num_samples)
            interp_heater_data_list = []
            for i in range(10):
                interp_heater_data = self._interp_funcs[i](sample_times)
                interp_heater_data_list.append(interp_heater_data)

            cls_data_list.append({
                "class": cls,
                "sample_times": sample_times,
                "sample_vals": interp_heater_data_list
            })

        return cls_data_list

In [38]:
s = SensorCls(1, 1, 0, sensor_data, labels, interp_funcs)
cls_list = s.get_interpolated_data()

In [56]:
print(cls_list[0]["sample_times"][0])
print(cls_list[1]["sample_times"][0])
print(cls_list[2]["sample_times"][0])
print(cls_list[3]["sample_times"][0])

610606.0
2010330.0
3510828.0
5310933.0


In [61]:
def plot_cls(cls_list, day, mat, sensor):
    title = f"Day {day}, Matrix {mat}, Sensor {sensor}"
    fig = go.Figure()

    for cls in cls_list:
        for i in range(10):
            fig.add_trace(go.Scatter(x=cls["sample_times"],
                                     y=cls["sample_vals"][i],
                                     mode="lines",
                                     name=f"H.S. {i}"))

    # for label in labels[f"day{day}"][f"mat{mat}"]:
    #     fig.add_vrect(x0=label["start"],
    #                   x1=label["end"],
    #                   annotation_text=classes[label["label"]],
    #                   annotation_position="top left",
    #                   fillcolor=colors[label["label"]],
    #                   opacity=0.2,
    #                   line_width=0)

    fig.update_layout(title=title, title_x=0.5, width=1000, height=600)

    fig.update_yaxes(type="log")
    fig.show()

In [62]:
plot_cls(cls_list, 1, 1, 0)